# $\left\langle L_{B,\tau, Z}\right\rangle$ Calculation

Here we want to provide the code to calculate $\left\langle L_{B,\tau, Z}\right\rangle$ for a given set of isochrhones. Since this can be an expensive task, we're going to use Cython to speed up some parts of the code. Much of this notebook is identical to the notebook which calculates $f_{\rm det}$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
%load_ext Cython

In [2]:
# First specify the properties of the populations you'd like to calculate f_detect for
# by giving lists of metallicities and ages. These should be linearly spaced 
metlist = np.array([0.5,0.25,0.0,-0.25,-0.5,-0.75,-1.0,-1.25,-1.5,-1.75,-2.0,-2.25,-2.5,-2.75,-3.,-3.25])
agelist = np.array([8.95,9.,9.05,9.1,9.15,9.2,9.25,9.3,9.35,9.4,9.45,9.5,9.55,9.6,9.65,9.7,9.75,9.8,9.85,9.9,9.95,10.,10.05,10.1,10.15])
nmet = len(metlist) # number of metallicities
nage = len(agelist) # number of ages
dage = agelist[1] - agelist[0] # logarithmic difference in  age bins, for selection below

## specify directory where isochrones are stored, this is set up for the 
## MIST Roman Isocrones, changing that would involve changing the below
iso_dir = "../mist_isos/"
## name od output file
output_name = "data/tot_fdet" # it will have a .npy at the end

## define the IMF normalization
## specify the power-law index for the IMF that you would like to use
mpow = -1.3
## specify the number of mass samples you would like for integrals to be 
## numericall performed below
nmass_samp = 100000000
# mass range in solar masses
(mMin,mMax) = (0.1,100)
# masses spaced linearly throughout imf space
mlin = np.linspace(mMin,mMax,nmass_samp)
# the IMF normalization over the mass range of interest
xim_norm = np.trapz(mlin**mpow,mlin)

# specify the number of filters. The default is 6 (Z,Y,J,H,F). Changing this 
# number will require some hard code changes below
nfilts = 5

The following cell is where we define the function needed to compute the average luminosity, which is effectively the only cell which is different from the $f_{\rm det}$ calculation

In [6]:
%%cython
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True
import numpy as np
cimport numpy as np


cimport cython

cpdef double get_Lavg(const double[::1] B_mu, const double[::1] ms, 
                      const double[::1] xiofm, const int N):
    cdef:
        double fsum = 0
        double aB,bB,aX,bX,mcross,xicross
        int i
    
    for i in range(N-1):
        fsum += (10**(B_mu[i]/-2.5)*xiofm[i] + 10**(B_mu[i+1]/-2.5)*xiofm[i+1])*(ms[i+1] - ms[i])/2.
    return fsum

In [7]:
%%time
# shape of the total output array
totdat = np.zeros((nmet,nage,nfilts))
for (l,met) in enumerate(metlist):
    print(met)
    if met>-0.1:
        (age,imass,T,L) = np.loadtxt("%smist_fehp%1.2f.iso.cmd"%(iso_dir,abs(met)),usecols=[1,2,4,6]).T
        (Z,Y,J,H,F) = np.loadtxt("%smist_fehp%1.2f.iso.cmd"%(iso_dir,abs(met)),usecols=[10,11,12,14,15]).T
    else:
        (age,imass,T,L) = np.loadtxt("%smist_fehm%1.2f.iso.cmd"%(iso_dir,abs(met)),usecols=[1,2,4,6]).T
        (Z,Y,J,H,F) = np.loadtxt("%smist_fehm%1.2f.iso.cmd"%(iso_dir,abs(met)),usecols=[10,11,12,14,15]).T

    (Zage,Yage,Jage,Hage,Fage) = (np.zeros(nage),np.zeros(nage),np.zeros(nage),np.zeros(nage),np.zeros(nage))
    for (k,a) in enumerate(agelist):
        sids = np.intersect1d(np.where(age>a-dage/2),np.where(age<a+dage/2))

        # interpolate what the right magnitudes are
        AB_Vega = np.array([0.487, 0.653, 0.958, 1.287, 1.552]) #This is m_AB - m_Vega
        Z_out = Z[sids] + AB_Vega[0]
        Y_out = Y[sids] + AB_Vega[1]
        J_out = J[sids] + AB_Vega[2]
        H_out = H[sids] + AB_Vega[3]
        F_out = F[sids] + AB_Vega[4]
        mlin = imass[sids]
        xim = (imass[sids]**mpow)/xim_norm
        Zage[k] = get_Lavg(Z_out,mlin,xim,len(xim))
        Yage[k] = get_Lavg(Y_out,mlin,xim,len(xim))
        Jage[k] = get_Lavg(J_out,mlin,xim,len(xim))
        Hage[k] = get_Lavg(H_out,mlin,xim,len(xim))
        Fage[k] = get_Lavg(F_out,mlin,xim,len(xim))

    totdat[l] = np.array([Zage,Yage,Jage,Hage,Fage]).T


0.5
0.25
0.0
-0.25
-0.5
-0.75
-1.0
-1.25
-1.5
-1.75
-2.0
-2.25
-2.5
-2.75
-3.0
-3.25
CPU times: user 32.8 s, sys: 1.01 s, total: 33.8 s
Wall time: 34.1 s


In [9]:
np.save("data/tot_lum",totdat)